In [1]:
using LinearAlgebra, Random, Statistics

In [2]:
"""
    tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
Solve the tridiagonal system of linear equations described by the tridiagonal
matrix `M` with right-hand-side `g` assuming one of the eigenvalues is zero
(which results in a singular matrix so the general Thomas algorithm has been
modified slightly).
Reference CPU implementation per Numerical Recipes, Press et. al 1992 (§ 2.4)
"""
function tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
    N = length(f)
    ϕ = similar(f)
    γ = similar(f)

    β    = M.d[1]
    ϕ[1] = f[1] / β

    for j = 2:N
        γ[j] = M.du[j-1] / β
        β    = M.d[j] - M.dl[j-1] * γ[j]

        # This should only happen on last element of forward pass for problems
        # with zero eigenvalue. In that case the algorithmn is still stable.
        abs(β) < 1.0e-12 && break

        ϕ[j] = (f[j] - M.dl[j-1] * ϕ[j-1]) / β
    end

    for j = 1:N-1
        k = N-j
        ϕ[k] = ϕ[k] - γ[k+1] * ϕ[k+1]
    end

    return ϕ
end

tridiag

In [3]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [6]:
zF = [1, 2, 4, 7, 11, 16, 22, 29, 37]
Nz = length(zF) - 1
zF, zC, ΔzF, ΔzC = grid(zF)

([1, 2, 4, 7, 11, 16, 22, 29, 37], [1.5, 3.0, 5.5, 9.0, 13.5, 19.0, 25.5, 33.0], [1, 2, 3, 4, 5, 6, 7, 8], [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])

In [7]:
ld = [1/ΔzF[k] for k in 1:Nz-1]
ud = copy(ld)

7-element Array{Float64,1}:
 1.0                
 0.5                
 0.3333333333333333 
 0.25               
 0.2                
 0.16666666666666666
 0.14285714285714285

In [9]:
δ(k) = -1/ΔzF[k-1] - 1/ΔzF[k]
d = [-1/ΔzF[1], [δ(k) for k in 2:Nz]...]

8-element Array{Float64,1}:
 -1.0                
 -1.5                
 -0.8333333333333333 
 -0.5833333333333333 
 -0.45               
 -0.3666666666666667 
 -0.30952380952380953
 -0.26785714285714285

In [10]:
M = Tridiagonal(ld, d, ud)
Base.print_matrix(IOContext(stdout, :limit => false), M)

 -1.0   1.0    ⋅                    ⋅                    ⋅      ⋅                     ⋅                     ⋅                 
  1.0  -1.5   0.5                   ⋅                    ⋅      ⋅                     ⋅                     ⋅                 
   ⋅    0.5  -0.8333333333333333   0.3333333333333333    ⋅      ⋅                     ⋅                     ⋅                 
   ⋅     ⋅    0.3333333333333333  -0.5833333333333333   0.25    ⋅                     ⋅                     ⋅                 
   ⋅     ⋅     ⋅                   0.25                -0.45   0.2                    ⋅                     ⋅                 
   ⋅     ⋅     ⋅                    ⋅                   0.2   -0.3666666666666667    0.16666666666666666    ⋅                 
   ⋅     ⋅     ⋅                    ⋅                    ⋅     0.16666666666666666  -0.30952380952380953   0.14285714285714285
   ⋅     ⋅     ⋅                    ⋅                    ⋅      ⋅                    0.14285714285714285  -0.26

In [13]:
R = rand(MersenneTwister(0), Nz)
R .= R .- mean(R)

8-element Array{Float64,1}:
  0.4900442230630878 
  0.5767532530121119 
 -0.16903748678063935
 -0.15627443844806   
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [14]:
g = similar(R)
g[1:Nz-1] = ΔzC .* R[1:Nz-1]
g[Nz] = ΔzC[Nz-1] * R[Nz]
g

8-element Array{Float64,1}:
  0.7350663345946318
  1.4418831325302797
 -0.5916312037322378
 -0.7032349730162699
 -0.3009774657071796
 -0.8458237246706124
 -2.184762142367212 
 -1.9900052205650984

In [15]:
ϕ = tridiag(M, g)

8-element Array{Float64,1}:
 37.97176745618404 
 38.706833790778674
 43.060732725028494
 47.81668751520651 
 51.34502067671211 
 54.2505498000582  
 52.66224240004984 
 35.515882103469615

In [16]:
∇²ϕ = zeros(Nz)

@inline δz_aac(k, f) = @inbounds f[k+1] - f[k]
@inline ∂z_aac(k, ΔzF, f) = δz_aac(k, f) / ΔzF[k]

∇²ϕ[1] = ∂z_aac(1, ΔzF, ϕ) / ΔzC[1]
for k in 2:Nz-1
    ∇²ϕ[k] = (∂z_aac(k, ΔzF, ϕ) - ∂z_aac(k-1, ΔzF, ϕ)) / ΔzC[k]
end
∇²ϕ[Nz] = (∂z_aac(Nz, ΔzF, ϕ) - ∂z_aac(Nz-1, ΔzF, ϕ)) / ΔzC[Nz-1]
∇²ϕ

8-element Array{Float64,1}:
  0.49004422306308965 
  0.5767532530121102  
 -0.16903748678063937 
 -0.15627443844806047 
 -0.054723175583123916
 -0.13012672687240184 
 -0.2913016189822947  
 -0.2653340294086798  

In [17]:
R

8-element Array{Float64,1}:
  0.4900442230630878 
  0.5767532530121119 
 -0.16903748678063935
 -0.15627443844806   
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [18]:
∇²ϕ ≈ R

true